Things I added:  
pip install googletrans
pip install textblob

In [ ]:
#Run docker container
sudo docker run -d --name sparkbook -p 8881:8888 -v "$PWD":/home/jovyan/work jupyter/pyspark-notebook start.sh jupyter lab --LabApp.token=''

#Exec docker container
docker exec -it sparkbook

#Run Spark
#go on browser
localhost:8881

In [20]:
#Run1
%matplotlib inline

import os
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from datetime import datetime
from dateutil.parser import parse

plt.style.use('ggplot')

In [ ]:
#Run2
import pyspark as ps

spark = (ps.sql.SparkSession
         .builder
         .master('local[4]')
         .appName('Instacart')
         .getOrCreate()
        )

sc = spark.sparkContext
sc

In [64]:
#Run5
#Creates a text label above each bar in *rects*, displaying its height.
def autolabel(rects, orientation='vert'):
    #prints value above vertical bars
    if orientation=='vert':
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{}'.format(int(height)),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        weight = 'bold',
                        textcoords="offset points",
                        ha='center', va='bottom',
                        size=15)
    #Prints value to the right of horizontal bars
    elif orientation=='hort':
        for rect in rects:
            width = rect.get_width()
            ax.annotate('{}'.format(int(width)),
                        xy=(width, rect.get_y() + rect.get_height() / 2),
                        xytext=(3,-6),
                        textcoords="offset points",
                        size=15)

In [65]:
#Run6
#Creates a text label above each bar in *rects*, displaying its height.
def autolabel_percent(rects, orientation='vert'):
    #Prints percentage above bars for vertical bars
    if orientation=='vert':
        for rect in rects:
            height = rect.get_height()
            ax.annotate('{0:.2f}%'.format(height),
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),  # 3 points vertical offset
                        weight = 'bold',
                        textcoords="offset points",
                        ha='center', va='bottom',
                        size=15)
    #Prints percentages to the right of bars in horizontal bars
    elif orientation=='hort':
        for rect in rects:
            width = rect.get_width()
            ax.annotate("{0:.2f}%".format(width),
                        xy=(width, rect.get_y() + rect.get_height() / 2),
                        xytext=(3,-6),
                        textcoords="offset points",
                        size=15)

In [63]:
#Run7
#Print functions for bar graphs specifically
def print_bar(x, y, title, x_label, y_label, title='insert title', orientation='vert', color='blue', 
              width=0.65, fig_size=(23,7), percentage=False, weight='bold', tick_size=20, title_size=30):
    fig, ax = plt.subplots(figsize=fig_size)
    
    if orientation=='vert':
        bars_for_annotation = ax.bar(x, y, color=color, align='center', width=width)
    elif orientation=='hort':
        bars_for_annotation = ax.barh(x, y, color=color, align='center')
        
    plt.xticks(size = 13, rotation=90)
    plt.yticks(size = 15)
    plt.xlabel(x_label, size=tick_size, color=color)
    plt.ylabel(y_label, size=tick_size, color=color)
    plt.title(title, fontsize=title_size, color=color)
        
    if percentage:
        autolabel_percent(bars_for_annotation, orientation)
    elif not percentage:
        autolabel(bars_for_annotation, orientation)
    
    plt.show()

In [ ]:
#TODO: test for stemming

In [ ]:
#TODO: preprocessing the data (uncomment later)
useless_listing_columns = ['square_feet','country','listing_url','scrape_id','last_scraped','experiences_offered', 'license', 
                   'xl_picture_url','host_url','host_name','host_thumbnail_url','street','host_listings_count',
                   'neighbourhood_group_cleansed','state','market', 'calendar_last_scraped','host_picture_url',
                   'host_acceptance_rate', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
                   'maximum_maximum_nights', 'jurisdiction_names', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
                   'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms','maximum_nights_avg_ntm',
                   'has_availability','country_code','host_acceptance_rate','thumbnail_url','medium_url','weekly_price','monthly_price',
                   'calendar_updated','is_business_travel_ready', 'interaction', 'interaction', 'house_rules']

def preprocessing(path):
    #load the LISTINGS csv
    processed_listings_df = pd.read_csv(path+'listings.csv')
    #drop useless columns
    processed_listings_df.drop(columns = useless_listing_columns, inplace=True)
    
    #load the CALENDAR csv
    processed_calendar_df = pd.read_csv(path+'calendar.csv')
    #Convert date strings to datetime type
    date_time = pd.Series([datetime.strptime(x, '%Y-%m-%d') for x in processed_calendar_df['date']])
    processed_calendar_df['date'] = date_time
    
    #load the REVIEWS csv
    processed_reviews_df = pd.read_csv(path+'reviews.csv')
    #Drop useless columns. Id of review and the reviewer's name
    processed_reviews_df.drop(columns=['id', 'reviewer_name'], inplace=True)

    return processed_listings_df, processed_calendar_df, processed_reviews_df

In [2]:
#Load up listings
paris_listings = pd.read_csv('Data/Paris/listings.csv', nrows=50000)

/Users/winrichsy/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [197]:
#drop useless columns (Paris)
useless_columns = ['square_feet','country','listing_url','scrape_id','last_scraped','experiences_offered', 'license', 
                   'xl_picture_url','host_url','host_name','host_thumbnail_url','street','host_listings_count',
                   'neighbourhood_group_cleansed','state','market', 'calendar_last_scraped','host_picture_url',
                   'host_acceptance_rate', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights',
                   'maximum_maximum_nights', 'jurisdiction_names', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes',
                   'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms','maximum_nights_avg_ntm',
                   'has_availability','country_code','host_acceptance_rate','thumbnail_url','medium_url','weekly_price','monthly_price',
                   'calendar_updated','is_business_travel_ready', 'interaction', 'interaction', 'house_rules', 'experiences_offered']
paris_listings_shortened = paris_listings.drop(columns = useless_columns)

In [5]:
#save new csv
singapore = 'Data/Singapore/'
london = 'Data/London/'
paris = 'Data/Paris/'

# paris_data_shortened.to_csv (paris+'shortened_listings.csv', index = None, header=True)

In [ ]:
#Run3
paris_listings = spark.read.csv(paris +'shortened_listings.csv',
                           header = True,
                           sep = ",",
                           inferSchema = True)

paris_calendar = spark.read.csv(paris+'shortened_calendar.csv',
                            header = True,
                            sep = ",",
                            inferSchema = True)

paris_reviews = spark.read.csv(paris+'shortened_reviews.csv',
                                     header = True,
                                     sep = ",",
                                     inferSchema = True)

#===========================

In [44]:
#Load up Calendar (Paris)
paris_calendar = pd.read_csv(paris + 'calendar.csv', nrows=25000)

#Convert date strings to datetime type
date_time = pd.Series([datetime.strptime(x, '%Y-%m-%d') for x in paris_calendar['date']])
paris_calendar['date'] = date_time

In [143]:
#Load up Reviews (Paris)
paris_reviews = pd.read_csv(paris + 'reviews.csv', nrows=25000)

#Drop useless columns. Id of review and the reviewer's name
paris_reviews.drop(columns=['id', 'reviewer_name'], inplace=True)

In [184]:
#detect language of reviewers
#https://pypi.org/project/langdetect/
import langdetect as ld
#remove the rows with blank comments
paris_reviews = paris_reviews.loc[paris_reviews['comments'].notna()]
# paris_reviews = paris_reviews.loc[paris_reviews['comments']!='.']
paris_reviews = paris_reviews.loc[paris_reviews['comments'].str.len()>20]


comment_language = pd.Series([ld.detect(x) for x in paris_reviews['comments']])
paris_reviews['language'] = comment_language

#CAN: explain removing comments with less than 10 letters. it also finds the comments with random symbols

In [ ]:
paris_reviews.to_csv()

In [224]:
#https://pypi.org/project/langua/
from langua import Predict
p = Predict()

In [ ]:
#https://pypi.org/project/langua/
from langua import Predict
p = Predict()

#detect language of hosts
#1. Get only the description that are not empty
#2. Get descriptions that are more than 20, overstepping random symbols
paris_listings_shortened = paris_listings_shortened.loc[paris_listings_shortened['description'].notna()]
paris_listings_shortened = paris_listings_shortened.loc[paris_listings_shortened['description'].str.len()>20]

listing_language = [p.get_lang(x) for x in paris_listings['description']]

paris_listings_shortened['language'] = listing_language
#DO NOT RUN AGAIN UNTIL THE END: TAKES FOREVER

In [ ]:
#LEAVE THIS COMMENTED
# import pycountry
#Tested to change ISO 639-1 codes into their actual names
#listing_language = ['zh' if x=='zh-cn' or x=='zh-tw' else x for x in listing_language]

# listing_language = pd.Series([languages.get(alpha2=ld.detect(x)).name for x in listing_language[:10]])
# listing_language

# listing_language = pd.Series([pycountry.languages.get(alpha_2=ld.detect(x)).name for x in listing_language[:10]])
# listing_language

In [ ]:
import os
os.system('say "shit is done yo. get back to work"')

In [ ]:
listing_language.to_csv()

In [204]:
#https://pypi.org/project/iso-639/
from iso639 import languages

#Create dictionary on types of languages available
keys = ['af', 'ar', 'bg', 'bn', 'ca', 'cs', 'cy', 'da', 'de', 'el', 'en', 
        'es', 'et', 'fa', 'fi', 'fr', 'gu', 'he', 'hi', 'hr', 'hu', 'id', 
        'it', 'ja', 'kn', 'ko', 'lt', 'lv', 'mk', 'ml', 'mr', 'ne', 'nl', 
        'no', 'pa', 'pl', 'pt', 'ro', 'ru', 'sk', 'sl', 'so', 'sq', 'sv', 
        'sw', 'ta', 'te', 'th', 'tl', 'tr', 'uk', 'ur', 'vi', 'zh']

#convert iso-639 to iso language name
iso_name = [languages.get(alpha2=x).name for x in keys]

language_dict = {}
for idx, val in enumerate(keys):
    language_dict[val] = iso_name[idx]